In [ ]:
using TensorOperations, TensorKit
import TensorKit.×
using FLoops
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./models/Hubbard_Z2SU2.jl")
# include("./models/tJ_Z2.jl")
@show pkgversion(TensorKit)
using JLD2, HDF5
using Strided
Strided.enable_threads()

In [ ]:
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8.0
para[:μ] = 4.0
para[:pspace] = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
Hij = get_op_Hubbard("hij", para)
@show Hij

Hijp = permute(Hij, ((2, 1), (4, 3)))
@show Hijp
@assert Hij == Hijp
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./models/tJ_Z2.jl")
include("./simple_update_Fermionic/simple_update.jl")
include("./fast_full_update_Fermionic/fast_full_update.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")

ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)

Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, ["Δₛ", "Δₛdag"], para; site1=[1, 1], site2=[0, 2], get_op=get_op_tJ)

nothing

In [ ]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")


In [1]:
using TensorKit
using Zygote

In [23]:
function f(A::TensorMap, B::TensorMap)
    # _, R = leftorth(A, ((1,), (2,)))
    U, S, Vdag, ϵ = tsvd(A, ((1,), (2,)); trunc=notrunc(), alg=TensorKit.SVD())
    # @tensor Ap[a, b] := A[a, c] * A[c, b]
    return @tensor C = Vdag[a, c] * B[c, a]
end

f (generic function with 1 method)

In [24]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂×U₁]((0, 0) => 2, (1, 1) => 1)
# pspace = ℝ^3
A = TensorMap(randn, Float64, pspace, pspace)
B = TensorMap(randn, Float64, pspace, pspace)

# 内含 QR, SVD, 以及复杂矩阵乘的函数，无论 pspace 对称性如何，都可以直接求梯度
grd = gradient((a, b) -> f(a, b), A, B)
@show grd
nothing

grd = (TensorMap(Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)):
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()):
 0.10657041107350512   0.28900136391158776
 0.4184068356300867   -0.43901341639976077
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()):
 0.0
, TensorMap(Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)):
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()):
 0.8005859465140892   0.5992179421914368
 0.5992179421914368  -0.8005859465140892
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()):
 1.0000000000000004
)
